# Lama-2-7b finetuning

*Issues with the code below*

Incorrect LoRA Application, the error suggests that the model might not be correctly configured to track gradients for the LoRA weights.

Tokenization or Data Formatting Issues, tokenizing and preparing  dataset leads to the error. If the tokenized data or labels aren't in the expected format or if there are inconsistencies, it can disrupt the gradient flow.

Model Device Placement, although the model moved explicitly to the GPU using .cuda(), there might be intermediate tensors or operations that are not on the correct device, leading to issues with gradient computation.

Training Arguments or Optimizer Misconfiguration, incompatible training arguments or optimizer settings can sometimes interfere with gradient tracking.


In [ ]:
!pip install transformers accelerate peft bitsandbytes
!pip install datasets
!pip install huggingface_hub
!huggingface-cli login

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments, Trainer,BitsAndBytesConfig
from datasets import Dataset
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Load dataset from Google CSV
spreadsheet_url = 'https://drive.google.com/...'
import gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Open and convert to DataFrame
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.get_worksheet(0)
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])


Mounted at /content/drive


In [ ]:
# Prepare fine-tuning data
fine_tuning_data = []
for _, row in df.iterrows():
    prompt, response = row['user_input'], row['chatbot_response']
    if isinstance(prompt, str) and isinstance(response, str):
        fine_tuning_data.append({'prompt': prompt, 'response': response})

fine_tuning_df = pd.DataFrame(fine_tuning_data)

dataset = Dataset.from_pandas(fine_tuning_df)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# Load dataset
dataset = Dataset.from_pandas(fine_tuning_df)

# Split into train/test
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Model ID
model_id = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map=None,  # Disabled automatic device mapping
    low_cpu_mem_usage=True
)



# Function Loads the base model, applies LoRA, and moves to GPU.
def load_and_prepare_model():
    model = AutoModelForCausalLM.from_pretrained(  # Use AutoModelForSeq2SeqLM
        model_id,
        torch_dtype=torch.float16,
        device_map=None,  # Disabled automatic device mapping
        low_cpu_mem_usage=True
    ).cuda()  # Explicitly move to GPU

    # Apply LoRA to the model
    model = get_peft_model(model, lora_config)

    return model  # Return the prepared model


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="llama2_chatbot_finetuning",
    per_device_train_batch_size=1,  # Reduced batch size
    gradient_accumulation_steps=2,  # Reduced gradient accumulation steps
    gradient_checkpointing=True,    # Enabled gradient checkpointing
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="epoch",
    save_total_limit=2,
)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Applied LoRA to the model
model = get_peft_model(model, lora_config)

# View trainable parameters
print(model.print_trainable_parameters())

# Tokenizer padding token assignment
tokenizer.pad_token = tokenizer.eos_token

# Tokenize datasets
def tokenize_function(examples):
    tokenized_examples = tokenizer(
        examples["prompt"],
        text_target=examples["response"],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    labels = tokenized_examples["input_ids"]
    tokenized_examples["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in labels
    ]
    return tokenized_examples

# Tokenize train and eval datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["prompt", "response"]
)

tokenized_eval_dataset = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["prompt", "response"]
)

# Debug tokenized datasets
print(tokenized_train_dataset[0])

# Verify model trainable parameters
model = load_and_prepare_model()
for name, param in model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

torch.autograd.set_detect_anomaly(True)
trainer.train()




# Code changed  using TRL for RLHF Chatbots
- TRL is a powerful framework specifically designed for applying reinforcement learning to large language models (LLMs).

Supervised Fine-tuning (SFT): Fine-tuning a pre-trained LLM on a dataset of human-generated conversations. This initial step provides a foundation for the chatbot's behavior.

Reward Model Training: Train a separate reward model to score the chatbot's responses based on their quality, helpfulness, and relevance. This reward model guides the RL process.



In [ ]:
!pip install trl

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from datasets import Dataset

# Load your fine-tuned dataset
dataset = Dataset.from_pandas(fine_tuning_df)

# Model ID for LLaMA 2 or Flan-T5
model_id = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map=None,
    low_cpu_mem_usage=True
).cuda()

# Define PPO configuration
ppo_config = PPOConfig(
    model_name=model_id,
    learning_rate=1e-5,  # Adjust
    batch_size=1,
    forward_batch_size=1,
    ppo_epochs=4,
)

# Create PPOTrainer
trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=None,  # reference model for comparison
    tokenizer=tokenizer,
    dataset=dataset,
)

# Reward function
def reward_fn(samples, **kwargs):
    # reward logic here
    return torch.tensor([1.0] * len(samples))

# Training loop
for epoch in range(ppo_config.ppo_epochs):
    for batch in trainer.dataloader:
        query_tensors = batch["input_ids"].cuda()
        response_tensors = trainer.generate(query_tensors)

        # Calculate rewards for the generated responses
        rewards = reward_fn(response_tensors.cpu().numpy().tolist())

        # Update the policy using PPO
        stats = trainer.step(query_tensors, response_tensors, rewards)
        trainer.log_stats(stats, batch, rewards)

trainer.save_pretrained("./rlhf_chatbot")

# Training 2 models for comparison with Flan-t5-large model

In [ ]:
!pip install transformers accelerate peft bitsandbytes datasets huggingface_hub torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install gcsfs==2024.10.0
!pip install fsspec==2024.10.0

In [ ]:
!pip install transformers accelerate peft bitsandbytes
!pip install datasets
!pip install huggingface_hub
!huggingface-cli login
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())


In [ ]:
!pip install fsspec==2024.9.0

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TrainingArguments, Trainer,BitsAndBytesConfig
from datasets import Dataset
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Load dataset from Google CSV
spreadsheet_url = 'https://docs.google.com/...'
import gspread
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Open and convert to DataFrame
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.get_worksheet(0)
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

# Prepare fine-tuning data
fine_tuning_data = []
for _, row in df.iterrows():
    prompt, response = row['user_input'], row['chatbot_response']
    if isinstance(prompt, str) and isinstance(response, str):
        fine_tuning_data.append({'prompt': prompt, 'response': response})

fine_tuning_df = pd.DataFrame(fine_tuning_data)

dataset = Dataset.from_pandas(fine_tuning_df)


Mounted at /content/drive


In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType # TaskType
from datasets import Dataset

# Load dataset
dataset = Dataset.from_pandas(fine_tuning_df)

# Split dataset
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Model ID
model_id = "google/flan-t5-large"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Load and prepare the model with LoRA
def load_and_prepare_model():
  model = AutoModelForSeq2SeqLM.from_pretrained( #Seq2SeqLM
      model_id,
      torch_dtype=torch.float16,
      device_map=None,  # Disabled automatic device mapping
      low_cpu_mem_usage=True
  ).cuda()  # use GPU
  return get_peft_model(model, lora_config)


# Define training arguments for the 1 training
training_args_1 = TrainingArguments(
    output_dir="./results",
    run_name="llama2_chatbot_finetuning_1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="epoch",
    save_total_limit=2,
)

# Define training arguments for the second training loop
training_args_2 = TrainingArguments(
    output_dir="./results_2",
    run_name="llama2_chatbot_finetuning_2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="epoch",
    save_total_limit=2,
)

# Tokenizer padding token assignment
tokenizer.pad_token = tokenizer.eos_token

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(
        examples["prompt"],
        text_target=examples["response"],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    tokenized_examples["labels"] = tokenized_examples["input_ids"].copy()  # Set labels
    return tokenized_examples

# Tokenize train and eval datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["prompt", "response"]
)

tokenized_eval_dataset = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["prompt", "response"]
)


# Create separate models for each training loop
model_1 = load_and_prepare_model()  # Create a fresh model for the first loop
model_2 = load_and_prepare_model()  # Create another fresh model for the second loop

# Print trainable parameters
print(model_1.print_trainable_parameters())
print(model_2.print_trainable_parameters())  # Print for model_2 as well

# Fine-tune the model for the first training loop
trainer_1 = Trainer(
    model=model_1,
    args=training_args_1,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

trainer_1.train()


# Fine-tune the model for the second training loop
trainer_2 = Trainer(
    model=model_2,
    args=training_args_2,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

trainer_2.train() # training the second model

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.3004
None
trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.3004
None


<ipython-input-12-ad374fe09fe9>:114: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_1 = Trainer( # changed trainer variable name
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn